In [32]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score

In [33]:
train = pd.read_csv('train5.csv',sep='\t')
test = pd.read_csv('test5.csv',sep='\t')
train.sample(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,336,337,338,339,340,341,342,343,344,345
19806,19806,1,1,0,0,0,0,0,0.090909,0,...,0.450414,1,0,0,0,0.0,1,1,1,1
9994,9994,0,1,0,0,0,0,0,0.090909,0,...,0.109160,1,0,0,0,0.0,1,1,1,1
3065,3065,0,1,0,0,0,0,0,0.090909,0,...,0.450414,0,0,0,0,0.0,1,1,1,1
13199,13199,0,1,0,0,0,0,0,0.090909,0,...,0.450414,0,0,0,0,0.0,1,1,1,1
3612,3612,0,1,0,0,0,0,0,0.090909,0,...,0.450414,1,0,0,0,0.0,1,1,1,1


In [9]:
test.sample(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,336,337,338,339,340,341,342,343,344,345
969,969,NaN,1,0,0,0,0,0,0.090909,0,...,0.450414,1,0,0,0,0.000000,1,1,0,1
2643,2643,NaN,0,1,0,1,0,0,0.090909,0,...,0.195690,0,1,0,0,0.000000,1,1,1,0
532,532,NaN,1,0,0,0,0,0,0.090909,0,...,0.450414,1,0,0,0,0.000000,1,1,1,0
2321,2321,NaN,1,0,0,1,0,0,0.090909,0,...,0.260746,0,1,0,0,0.222222,1,1,1,1
3312,3312,NaN,1,0,0,1,0,0,0.090909,0,...,0.289893,0,0,1,0,0.000000,1,1,1,1


In [34]:
X = train.drop(['Unnamed: 0','0'], axis =1)
y = train['0']


In [35]:
jobs = 3
def grid_search(estimator, param, X, y):
    grid = GridSearchCV(estimator = estimator,
                    param_grid = param,
                   scoring = 'roc_auc',
                   n_jobs= jobs)
    grid.fit(X,y)
    return grid.best_score_ , grid.best_params_

In [24]:
estimator = LogisticRegression(class_weight = 'balanced')
param = {
    'C':[0.01, 0.03, 0.05, 0.07, 0.1],
    'solver': ['newton-cg', 'liblinear']
}

grid_search(estimator,param,X,y)

(0.7273309997135363, {'C': 0.05, 'solver': 'newton-cg'})

In [36]:
def to_csv(data, name):
    submission = pd.DataFrame.from_dict({'_ID_': test['Unnamed: 0'],
                                         '_VAL_' : test['0']})
    submission['_VAL_'] = data 
    submission.to_csv(str(name + '.csv'), index=False)

In [23]:
model = LogisticRegression(class_weight = 'balanced', C = 0.05, solver = 'newton-cg', n_jobs = 3)
model.fit(X,y)
data = model.predict(test.drop(['Unnamed: 0','0'], axis = 1))
to_csv(data, 'lg_7273_')

In [26]:
skf = StratifiedKFold(shuffle=True, random_state=42)

lg = LogisticRegression(class_weight= 'balanced')

parameters = {"base_estimator__C": [0.005, 0.001, 0.01]}

bg = BaggingClassifier(lg, random_state=42, n_estimators=25)

r_grid_search = RandomizedSearchCV(bg, parameters, scoring ='roc_auc', n_iter=5, cv=skf, random_state=1)
r_grid_search = r_grid_search.fit(X, y)

print(r_grid_search.best_score_)


0.7160642379719788


In [28]:
model = r_grid_search.best_estimator_ 
model.fit(X,y)
data = model.predict(test.drop(['Unnamed: 0','0'], axis = 1))
to_csv(data, 'lg_7160_')

In [44]:
estimator = RandomForestClassifier(n_estimators = 10)
skf = StratifiedKFold(shuffle=True, random_state=42)
estimator.fit(X,y)
results = cross_val_score(estimator, X, y, cv=skf)
results.mean()

0.823540957426803

In [45]:
data = estimator.predict(test.drop(['Unnamed: 0','0'], axis = 1))
to_csv(data, 'rf_8235_')

In [49]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB

In [54]:
clf1 = LogisticRegression(random_state=42)
clf2 = RandomForestClassifier(random_state=42)
clf3 = GaussianNB()

estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)]

eclf1 = VotingClassifier(estimators=estimators, voting='soft')
results = cross_val_score(estimator, X, y, cv=skf)
results.mean()

0.8213770423461927

In [56]:
eclf1.fit(X,y)
data = eclf1.predict(test.drop(['Unnamed: 0','0'], axis = 1))
to_csv(data, 'ans_8213_')

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
